<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Backpropagation Practice

## *Data Science Unit 4 Sprint 2 Assignment 2*

Implement a 3 input, 4 node hidden-layer, 1 output node Multilayer Perceptron on the following dataset:

| x1 | x2 | x3 | y |
|----|----|----|---|
| 0  | 0  | 1  | 0 |
| 0  | 1  | 1  | 1 |
| 1  | 0  | 1  | 1 |
| 0  | 1  | 0  | 1 |
| 1  | 0  | 0  | 1 |
| 1  | 1  | 1  | 0 |
| 0  | 0  | 0  | 0 |

If you look at the data you'll notice that the first two columns behave like an XOR gate while the last column is mostly just noise. Remember that creating an XOR gate was what the perceptron was criticized for not being able to learn. 

In [75]:
X = np.array(([0,0,1],
              [0,1,1],
              [1,0,1],
              [0,1,0],
              [1,0,0],
              [1,1,1],
              [0,0,0]), dtype=float)

y = np.array(([0],
              [1],
              [1],
              [1],
              [1],
              [0],
              [0]), dtype=float)

In [1]:
import numpy as np

In [76]:
def sigmoid(s):
    return 1 / (1+np.exp(-s))
    
def sigmoidPrime(s):
    return s * (1 - s)

In [77]:
weights1 = np.random.rand(3, 4)
weights2 = np.random.rand(4, 1)

def output(X):
    weighted_sum1 = np.dot(X, weights1)
    activated1 = sigmoid(weighted_sum1)
    weighted_sum2 = np.dot(activated1, weights2)
    activated2 = sigmoid(weighted_sum2)
    return (activated1, activated2)

In [78]:
for _ in range(10000):
    (hidden, out) = output(X)
    out_error = y - out
    out_delta = out_error * sigmoidPrime(out)
    
    z2_error = np.dot(out_delta, weights2.T)
    z2_delta = z2_error * sigmoidPrime(hidden)
    
    weights1 += np.dot(X.T, z2_delta)
    weights2 += np.dot(hidden.T, out_delta)

In [79]:
np.mean(out_error)

0.0009793414872682493

In [80]:
np.round(output(X)[1])

array([[0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.]])

## Try building/training a more complex MLP on a bigger dataset.

Use the [MNIST dataset](http://yann.lecun.com/exdb/mnist/) to build the cannonical handwriting digit recognizer and see what kind of accuracy you can achieve. 

If you need inspiration, the internet is chalk-full of tutorials, but I want you to see how far you can get on your own first. I've linked to the original MNIST dataset above but it will probably be easier to download data through a neural network library. If you reference outside resources make sure you understand every line of code that you're using from other sources, and share with your fellow students helpful resources that you find.


### Parts
1. Gathering & Transforming the Data
2. Making MNIST a Binary Problem
3. Estimating your Neural Network (the part you focus on)

### Gathering the Data 

`keras` has a handy method to pull the mnist dataset for you. You'll notice that each observation is a 28x28 arrary which represents an image. Although most Neural Network frameworks can handle higher dimensional data, that is more overhead than necessary for us. We need to flatten the image to one long row which will be 784 values (28X28). Basically, you will be appending each row to one another to make on really long row. 

In [19]:
import numpy as np
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

In [20]:
# input image dimensions
img_rows, img_cols = 28, 28

In [21]:
# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11493376/11490434 [==============================] - 3s 0us/step


In [22]:
x_train = x_train.reshape(x_train.shape[0], img_rows * img_cols)
x_test = x_test.reshape(x_test.shape[0], img_rows * img_cols)

# Normalize Our Data
x_train = x_train / 255
x_test = x_test / 255

In [23]:
# Now the data should be in a format you're more familiar with
x_train.shape

(60000, 784)

### Making MNIST a Binary Problem 
MNIST is multiclass classification problem; however we haven't covered all the necessary techniques to handle this yet. You would need to one-hot encode the target, use a different loss metric, and use softmax activations for the last layer. This is all stuff we'll cover later this week, but let us simply the problem for now: Zero or all else.

In [24]:
import numpy as np

y_temp = np.zeros(y_train.shape)
y_temp[np.where(y_train == 0.0)[0]] = 1
y_train = y_temp

y_temp = np.zeros(y_test.shape)
y_temp[np.where(y_test == 0.0)[0]] = 1
y_test = y_temp

In [25]:
# A Nice Binary target for ya to work with
y_train

array([0., 1., 0., ..., 0., 0., 0.])

In [125]:
y_train = np.array([y_train]).T

### Estimating Your `net

In [118]:
weights1 = np.random.rand(784, 100)
bias1 = np.random.random()

weights2 = np.random.rand(100, 1)
bias2 = np.random.random()

def output(X):
    weighted_sum1 = np.dot(X, weights1) + bias1
    activated1 = sigmoid(weighted_sum1)
    weighted_sum2 = np.dot(activated1, weights2) + bias2
    activated2 = sigmoid(weighted_sum2)
    return (activated1, activated2)

In [126]:
# x_train is too large, so I'll split it up.
X = x_train[0:1000]
y = y_train[0:1000]

In [119]:
for _ in range(1000):
    (hidden, out) = output(X)
    
    # print("hidden", hidden.shape)
    # print("out", out.shape)
    
    out_error = y - out
    out_delta = out_error * sigmoidPrime(out)
    
    # print("out_error", out_error.shape)
    # print("out_delta", out_delta.shape)
    
    z2_error = np.dot(out_delta, weights2.T)
    z2_delta = z2_error * sigmoidPrime(hidden)
    
    # print(np.mean(out_delta))
    
    weights1 += np.dot(X.T, z2_delta)
    bias1 += np.mean(z2_error)
    
    weights2 += np.dot(hidden.T, out_delta)
    bias2 += np.mean(out_error)

print(np.mean(out_error))

0.09699999999995995


In [124]:
pred = np.round(output(x_train)[1])

In [123]:
from sklearn.metrics import accuracy_score

In [127]:
accuracy_score(pred, y_train)

0.9012833333333333

Surprisingly, this models doesn't seem to be overfitted, getting 90%+ accuracy on the full dataset despite only seeing 1/60th of all the examples.

## Stretch Goals: 

- Make MNIST a multiclass problem using cross entropy & soft-max
- Implement Cross Validation model evaluation on your MNIST implementation 
- Research different [Gradient Descent Based Optimizers](https://keras.io/optimizers/)
 - [Siraj Raval the evolution of gradient descent](https://www.youtube.com/watch?v=nhqo0u1a6fw)
- Build a housing price estimation model using a neural network. How does its accuracy compare with the regression models that we fit earlier on in class?